In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("..") # Adds higher directory to python modules path.
import branchingdnn as branching
from branchingdnn.utils import * 

In [4]:
#  x = tf.keras.models.load_model("../models/alexNetv5.hdf5")
    
x = tf.keras.models.load_model('../models/alexNetv6_feat_distill_3.hdf5', custom_objects={'confidenceScore': confidenceScore,
                                                                         'unconfidence': unconfidence,
                                                                         'confidenceDifference': confidenceDifference,
                                                                         'BranchEndpoint': branching.branches.branch.BranchEndpoint,
                                                                         'FeatureDistillation': branching.branches.branch.FeatureDistillation})

In [37]:
#process for self distilation
#add y_true as an input for the model, here called 'targets'. targets is not linked to the main model path
#targets is added as input at the model define call
#targets is used as an additional input to the endpoint layers
#in endpoint layers, perform the loss function using the prev_layer input and the 'targets'

#determine if the additional loss is precomputed or computed at the endpoint layer.

from tensorflow.keras import layers

inputs = keras.Input(shape=(227,227,3))
x = keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
# x = keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.Flatten()(x)
#### bottleneck
# x = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)

# x = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)
# x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)


In [38]:
model = keras.Model(inputs=inputs, outputs=[x], name="alexnet")
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
model.summary()


Model: "alexnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_15 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 13, 13, 96)        0         
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 6, 6, 96)          0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 6, 6, 256)         2483